# Projeto 1 - Ciência dos Dados

Nome: Matheus Pereira Belarmino

Nome: Augusto Ricardo Ferreira

Nome: Diogo Gingler Diniz

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\matheus\Desktop\codigos\Cdados_denovo_;(\projeto_1\Projeto1-Cdados-24


Carregando a base de dados com as mensagens dos seus arquivos:

In [3]:
train = pd.read_csv('dados_treino_TRIO_matheuspb.csv')
train.head(5)

,sentence,target
0,If the reasoning is from initial facts to an u...,HG
1,The use of information technology in governmen...,HG
2,"Additionally, players should always gamble res...",AI
3,Search filters: Make sure the job site has sea...,AI
4,Yes‚ these accounts were sometimes hidden‚ Mic...,HG


In [4]:
test = pd.read_csv('dados_teste_TRIO_matheuspb.csv')
test.head(5)

,sentence,target
0,com offers a search engine that emphasizes inf...,HG
1,Cyberpunk typically features protagonists who ...,AI
2,They are often used in data centers and other ...,AI
3,"In the late 1990s and early 2000s, the popular...",AI
4,"Stacks are commonly used in programming, espec...",AI


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
pd.read_csv('dados_teste_TRIO_matheuspb.csv')

,sentence,target
0,com offers a search engine that emphasizes inf...,HG
1,Cyberpunk typically features protagonists who ...,AI
2,They are often used in data centers and other ...,AI
3,"In the late 1990s and early 2000s, the popular...",AI
4,"Stacks are commonly used in programming, espec...",AI
...,...,...
625,EPS (Encapsulated PostScript) is a vector-base...,HG
626,Data that is highly secure may be more difficu...,AI
627,9 and V1.,HG
628,Collaboration: Allows users to collaborate on ...,AI


In [17]:
def limpa_texto(texto):
    caractere_especial = ['', '!', '@', '¨', '#', '$',  '%', '&', '*', '?', '"',',','.',"-","(",")"]
    texto_limpo = ''
    for letra in texto:
        if letra not in  caractere_especial:
            letra_nova = letra.lower()
            texto_limpo += letra_nova
    return texto_limpo

# limpeza do test
test.columns = ['sentence','target']
test['sentence'] = test['sentence'].apply(limpa_texto)
test['target'] = test['target'].apply(limpa_texto)
test['target'].value_counts()

# limpeza do train
train.columns = ['sentence','target']
train['sentence'] = train['sentence'].apply(limpa_texto)
train['target'] = train['target'].apply(limpa_texto)
train['target'].value_counts()

test['sentence']

0      com offers a search engine that emphasizes inf...
1      cyberpunk typically features protagonists who ...
2      they are often used in data centers and other ...
3      in the late 1990s and early 2000s the populari...
4      stacks are commonly used in programming especi...
                             ...                        
625    eps encapsulated postscript is a vectorbased r...
626    data that is highly secure may be more difficu...
627                                             9 and v1
628    collaboration: allows users to collaborate on ...
629    an emulator is a program that runs on one comp...
Name: sentence, Length: 630, dtype: object

In [12]:
train

,sentence,target
0,if the reasoning is from initial facts to an u...,hg
1,the use of information technology in governmen...,hg
2,additionally players should always gamble resp...,ai
3,search filters: make sure the job site has sea...,ai
4,yes‚ these accounts were sometimes hidden‚ mic...,hg
...,...,...
1465,1,ai
1466,this allows users to edit or repurpose the con...,ai
1467,some of that traffic may be under the administ...,hg
1468,they also explore the impact of technology on ...,ai


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens. Ex: stemming, lemmatization, stopwords
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**